In [1]:
with open('day21.txt') as f:
    codes = f.read().strip().split('\n')

In [2]:
numeric_pad = [['7', '8', '9'],
               ['4', '5', '6'],
               ['1', '2', '3'],
               ['N', '0', 'A']]


directional_pad = [['N', '^', 'A'],
                   ['<', 'v', '>']]

direction_maps = {
    (-1, 0): '^',
    (1, 0): 'v',
    (0, -1): '<',
    (0, 1): '>'
}

In [3]:
from collections import deque
from itertools import permutations
from collections import defaultdict, Counter

def bfs(start, pad, commands):
    queue = deque([(start[0], start[1], '')])
    visited = set()
    
    while queue:
        r, c, path = queue.popleft()
        command = pad[start[0]][start[1]] + pad[r][c]
        if command not in commands: commands[command] = path
        visited.add((r, c))
        
        for dr, dc in direction_maps:
            nr, nc = r + dr, c + dc
            if 0 <= nr < len(pad) and 0 <= nc < len(pad[0]) and (nr, nc) not in visited:
                queue.append((nr, nc, path + direction_maps[(dr, dc)]))
                
def get_commands(pad):
    commands = {}
    for r in range(len(pad)):
        for c in range(len(pad[0])):
            bfs((r, c), pad, commands)
    return commands

def get_permutations(commands):
    p_commands = defaultdict(list)
    for command in commands:
        if 'N' in command: continue
        for dir_code in permutations(commands[command]):
            dead_command = commands[command[0] + 'N']
            if Counter(dir_code[:len(dead_command)]) == Counter(dead_command):
                continue
            p_commands[command].append(''.join(dir_code) + 'A')
        p_commands[command] = list(set(p_commands[command]))
    return p_commands


In [4]:
numeric_commands = get_permutations(get_commands(numeric_pad))
directional_commands = get_permutations(get_commands(directional_pad))

dp = {0: {command : 1 for command in directional_commands}}

for depth in range(1, 26):
    dp[depth] = {}
    for command in directional_commands:
        min_code_len = float('inf')
        for dir_code in directional_commands[command]:
            initial_pos = 'A'
            code_len = 0
            for button in dir_code:
                code_len += dp[depth - 1][initial_pos + button]
                initial_pos = button
            min_code_len = min(min_code_len, code_len)
        dp[depth][command] = min_code_len

In [5]:
def get_dir_codes(numeric_code):
    initial_pos = 'A'
    candidate_codes = ['']
    for button in numeric_code:
        tmp_candidate_codes = []
        for candidate_code in candidate_codes:
            for candidate in numeric_commands[initial_pos + button]:
                tmp_candidate_codes.append(candidate_code + candidate)
        candidate_codes = tmp_candidate_codes
        initial_pos = button
    return candidate_codes

def get_code_length_dir_code(dir_code, depth):
    initial_pos = 'A'
    code_len = 0
    for button in dir_code:
        code_len += dp[depth][initial_pos + button]
        initial_pos = button
    return code_len

def get_code_length_numeric_code(numeric_code, depth):
    candidate_codes = get_dir_codes(numeric_code)
    return min(get_code_length_dir_code(candidate_code, depth) for candidate_code in candidate_codes)

# Part 1 and 2

In [6]:
res_2 = res_25 = 0
for numeric_code in codes:
    res_2 += get_code_length_numeric_code(numeric_code, 2) * int(numeric_code[:-1])
    res_25 += get_code_length_numeric_code(numeric_code, 25) * int(numeric_code[:-1])

print('Part 1:', res_2)
print('Part 2:', res_25)

Part 1: 238078
Part 2: 293919502998014
